<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preamble" data-toc-modified-id="Preamble-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preamble</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Classes" data-toc-modified-id="Classes-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Classes</a></span></li></ul></li><li><span><a href="#Main" data-toc-modified-id="Main-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Main</a></span></li></ul></div>

# Preamble

## Imports

In [1]:
import pandas as pd
import numpy as np
import time
import sys
sys.setrecursionlimit(10**6)

## Classes

In [29]:
class ConnectionEngine():
    def __init__(self,num_people=None,num_connections=None):
        self.num_people = num_people
        self.num_connections = num_connections

    def _build_connection_list(self,agent,population,num_connections):
        # Break counter
        #if _cnt == 0:
        #    _cnt += 1
        # Return IDs of people with connections less than num_connections
        available_to_connect = (
            lambda agent,population: population.drop(agent).query('num_connections < {}'
                                                                     .format(num_connections)
                                                                    ).index
        )

        ## Update number of connections
        #population['num_connections'] = population.connections.apply(len)
        # Get other agents available to connect
        available = available_to_connect(agent,population)
        # Randomly choose connection
        if len(available) > 0:
            connection = np.random.choice(available)
            # Make connection
            population.iloc[connection].connections.append(agent)
            population.iloc[agent].connections.append(connection)

            # Update number of connections
            population.iloc[[agent,connection],2] += 1
            #if _cnt < 10:
            while population.num_connections[agent] < num_connections:
                self._build_connection_list(agent,
                                       population,
                                       num_connections)

        return population
    
    def create_connections(self,verbose=False):
        num_connections = self.num_connections
        num_people = self.num_people
        population = pd.DataFrame(
            {
                'index': [i for i in range(num_people)],
                'connections': [[] for i in range(num_people)],
                'num_connections': [0 for i in range(num_people)]
            }
        )
        
        _update = num_people*0.1
        for _per in population.index:
            if verbose:
                if _per % _update == 0:
                    print('{:.0f}% complete'.format(_per/num_people*100))
            self._build_connection_list(_per,population,num_connections)
            
        self.population = population
        
        return population
            


# Main

For some reason, only works with function outside of class

In [17]:
def run_experiment(num_connections=None,base=None,start=1,stop=10):
    _data = {}
    for i in range(start,stop):
        num_people = i*base
        _start = time.time()
        _ = ConnectionEngine(num_people=num_people,num_connections=num_connections)
        _.create_connections()
        _end = time.time()
        _data[num_people] = {
            'num_connections' :num_connections,
            'time': _end-_start
        }
    
    return _data

In [37]:
%%time
connections = ConnectionEngine(num_people=10000,num_connections=10)

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 9.78 µs


In [38]:
%%time
connections.create_connections(verbose=False)

CPU times: user 3min 33s, sys: 7.17 s, total: 3min 40s
Wall time: 3min 41s


,index,connections,num_connections
0,0,"[8717, 2722, 7378, 8727, 686, 3479, 7071, 5302...",10
1,1,"[6444, 8462, 7485, 3991, 9335, 1666, 1606, 143...",10
2,2,"[3456, 8839, 3806, 3254, 2379, 6747, 6800, 637...",10
3,3,"[1840, 1374, 8839, 2390, 1821, 1638, 3270, 669...",10
4,4,"[9405, 2179, 9055, 6740, 6992, 8951, 812, 6672...",10
...,...,...,...
9995,9995,"[1599, 2332, 2576, 4103, 6286, 8174, 8336, 863...",10
9996,9996,"[712, 1132, 2536, 3717, 5988, 7187, 8079, 9236...",10
9997,9997,"[417, 662, 1562, 5292, 5541, 6039, 6227, 6660,...",10
9998,9998,"[26, 5115, 6884, 7212, 7316, 8555, 8632, 8744,...",10


In [36]:
sys.getsizeof(connections.population)/1024/1024

1.220855712890625

In [ ]:
agent = 2
connection = 5

In [ ]:
population.iloc[[agent,connection],2] += 1

In [ ]:
connections.population.head(10)


In [ ]:
%%time
data = run_experiment(num_connections=3,base=100,start=1,stop=5)

In [ ]:
!touch experiment.py

In [ ]:
_start = time.time()
_.create_connections()
_end = time.time()
print(float('{:.2f}'.format(_end-_start)))

In [ ]:
_.population